---
title: Julia Basics
format:
    html: default
    ipynb: default
---

### What is Julia

- developped at MIT on top of opensource technologies
    - linux / git / llvm
- syntax inspired by Matlab but:
    - more consistent
    - lots of features from high level languages
- everything is JIT-compiled
    - interpreted vs compiled treadeoff
    - -> very fast
    - most of the base library is written in Julia
- opensource/free + vibrant community


Some useful links from QuantEcon:

* [Julia cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
* [Julia-Matlab comparison](https://cheatsheets.quantecon.org/index.html)
* [Julia essentials](https://lectures.quantecon.org/jl/julia_essentials.html)
* [Vectors, arrays and matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

Excellent resources at: [julialang](https://julialang.org/learning/)
- checkout JuliaAcademy, it's free

### an example of what you shouldn't do in Matlab

How I learnt: interpreted code is slow, so vectorize your coe.

In [1]:
"""
Multplies I by J by K
"""
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end

stupid_loop

In [5]:
@time stupid_loop(1000,1000,1000)

  0.530100 seconds


1.0e9

In [ ]:
@time [ stupid_loop(1000,1000,i) for i =1:10]

Code is translated to LLVM code then to instructions for the processor. Note that processor instructions are shorter than LLVM code.

In [6]:
@code_llvm stupid_loop(10,10,10)

; Function Signature: stupid_loop(Int64, Int64, Int64)
;  @ /home/pablo/Teaching/polytechnique/eco309/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:4 within `stupid_loop`
define double @julia_stupid_loop_6611(i64 signext %"I::Int64", i64 signext %"J::Int64", i64 signext %"K::Int64") #0 {
top:
;  @ /home/pablo/Teaching/polytechnique/eco309/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl within `stupid_loop`
  %".I::Int64" = call i64 @llvm.smax.i64(i64 %"I::Int64", i64 0)
;  @ /home/pablo/Teaching/polytechnique/eco309/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:6 within `stupid_loop`
; ┌ @ range.jl:904 within `iterate`
; │┌ @ range.jl:681 within `isempty`
; ││┌ @ operators.jl:379 within `>`
; │││┌ @ int.jl:83 within `<`
      %0 = icmp slt i64 %"I::Int64", 1
; └└└└
  br i1 %0, label %L85, label %L16.preheader

L16.preheader:                                    ; preds = %top
  %".J::Int64" = call i64 @l

In [7]:
@code_native stupid_loop(10,10,10)

	.text
	.file	"stupid_loop"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3, 0x0                          # -- Begin function julia_stupid_loop_6783
.LCPI0_0:
	.quad	0x3ff0000000000000              # double 1
	.text
	.globl	julia_stupid_loop_6783
	.p2align	4, 0x90
	.type	julia_stupid_loop_6783,@function
julia_stupid_loop_6783:                 # @julia_stupid_loop_6783
; Function Signature: stupid_loop(Int64, Int64, Int64)
; ┌ @ /home/pablo/Teaching/polytechnique/eco309/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:4 within `stupid_loop`
# %bb.0:                                # %top
; │ @ /home/pablo/Teaching/polytechnique/eco309/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl within `stupid_loop`
	#DEBUG_VALUE: stupid_loop:I <- $rdi
	#DEBUG_VALUE: stupid_loop:J <- $rsi
	#DEBUG_VALUE: stupid_loop:K <- $rdx
	xor	eax, eax
	test	rdi, rdi
	vxorpd	xmm0, xmm0, xmm0
	cmovg	rax, rdi
; │ @ /home/pablo/Teaching/polytechnique/eco309/tuto

### Syntax Review

#### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [ ]:
# Assign the value 10 to the variable x
x = 10

In [ ]:
2 == 3

In [ ]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
ϵ = 1e-4

Default semantic is pass-by-reference:

In [ ]:
a = [1,2,3,4]
b = a
a[1] = 10
b

To work on a copy: `copy` or `deepcopy`

In [ ]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

In [ ]:
a .== b

In [ ]:
a === b

#### Basic types

In [ ]:
# for any object `typeof` returns the type
?typeof

In [ ]:
typeof(a)

#### Numbers

In [ ]:
y = 2 + 2

In [ ]:
-y

In [ ]:
0.34*23

In [ ]:
3/4

In [ ]:
# Scalar multiplication doesn't require *
3(4 - 2)

In [ ]:
x = 4
2x

In [ ]:
typeof(x)

In [ ]:
sizeof(a)

#### Booleans

Equality

In [ ]:
0 == 1

In [ ]:
2 != 3

In [ ]:
3 <= 4

Identity

In [ ]:
a = [34, 35]
b = [34, 35]
c = a

In [ ]:
c === a

In [ ]:
b === a

Boolean operator

In [ ]:
true && false

In [ ]:
true || false

In [ ]:
!true

#### Strings

In [ ]:
# Strings are written using double quotes
str = "This is a string"

In [ ]:
ch = 'k' # this is a character

In [ ]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

In [ ]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a)"

In [ ]:
println("It took me $(a) iterations")

In [ ]:
# String concatenation using *
"hello" * "world"

In [ ]:
println("hello ", "world")

#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [1]:
A = [1, 2]

2-element Array{Int64,1}:
 1
 2

In [2]:
sizeof(A)

16

In [ ]:
typeof(A) == Vector{Int64}

In [5]:
# vectors have one dimension: they are indexed by an integer
A[1]

1

2d arrays are also called matrices... and can be used for matrix multiplications.

In [15]:
B = [0.1 0.2 0.3; 4 5 6]

2×3 Array{Float64,2}:
 0.1  0.2  0.3
 4.0  5.0  6.0

In [14]:
B*B'

2×2 Array{Float64,2}:
 0.14   3.2
 3.2   77.0

Vectorized operations take a ., even comparisons:

In [16]:
B.*B

2×3 Array{Float64,2}:
  0.01   0.04   0.09
 16.0   25.0   36.0

In [17]:
B .* B .< B

2×3 BitArray{2}:
 1  1  1
 0  0  0

Elements are always accessed with square brackets:

In [22]:
B[1,2]

0.2

In [23]:
B[:,1]

2-element Array{Float64,1}:
 0.1
 4.0

In [24]:
B[:,1:end-1]

2×2 Array{Float64,2}:
 0.1  0.2
 4.0  5.0

#### Control flow

Conditions

In [25]:
x = -3
if x < 0
    println("x is negative")
elseif x > 0 # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

x is negative


While

In [29]:
i = 3
while i > 0
    println(i)
    i = i - 1
end

3
2
1


For loops

In [31]:
# Iterate through ranges of numbers
for i = 1:3
    println(i)
    
    if i == 2
        break
    end
end

1
2


In [34]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city in cities
    println(city)
end

Boston
New York
Philadelphia


In [41]:
for t in zip(cities, states)
    println(t)
end


("Boston", "MA")
("New York", "NY")
("Philadelphia", "PA")


In [42]:
# Iterate through arrays of tuples using zip
for (city, state) in zip(cities, states)
    println("$city, $state")
end

Boston, MA
New York, NY
Philadelphia, PA


In [43]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City number $i is $city")
end

City number 1 is Boston
City number 2 is New York
City number 3 is Philadelphia


#### List comprehensions

In [44]:
[i^2 for i=1:10]

10-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [45]:
[i^2 for i=1:10 if mod(i,2)==0]

5-element Array{Int64,1}:
   4
  16
  36
  64
 100

### Function definitions

Basic functions

In [2]:
function f(a,b,c)
    res = a + b +c
    return res
end

f (generic function with 1 method)

In [3]:
f(3,4,3)

10

Optional arguments

In [4]:
function f(a,b,c=1)
    res = a + b +c
    return res
end

f (generic function with 2 methods)

In [5]:
f(1,2,3)

6

In [6]:
f(1,2)

4

Keyword arguments

In [19]:
function g(a,b,c; operator=(+), add_one=false)
    res = operator(a, operator( b , c) )
    if add_one
        res +=1
    end
    return res
end

g (generic function with 1 method)

In [15]:
g(1,2,3)

6

In [16]:
g(1.0,2.9,3.0; operator=(/))

1.0344827586206897

In [20]:
g(1.0,2.9,3.0; operator=(/), add_one=true)

2.0344827586206895

In [21]:
g(1.0,2.9,3.0; add_one=true, operator=(/), )

2.0344827586206895

One liners:

In [22]:
g(x,y) = x^2 + y^2

g (generic function with 2 methods)

Anonymous function:

In [28]:
fun = (x,y) -> x^2 + 1 + y

#12 (generic function with 1 method)

In [29]:
f(1,2,3; operator=fun)

10

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

In [30]:
struct Parameter_without_types
    value
    name
end

In [32]:
s = Parameter_without_types(4.4, "no type at all")

Parameter_without_types(4.4, "no type at all")

In [35]:
# Type definition
struct Parameter
    value::Float64
    transformation::Function # Function is a type!
    tex_label::String
    description::String
end

In [39]:
Parameter(4,fun,"\\Gamma", "My parameter")

Parameter(4.0, var"#12#13"(), "\\Gamma", "My parameter")

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [40]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\beta", "Discount rate")

Parameter(0.9, identity, "\beta", "Discount rate")

In [42]:
β.value

0.9

In [45]:
Parameter(value, transformation, tex) = Parameter(value, transformation, tex, "no description")

Parameter

In [46]:
Parameter(0.34, identity, "\beta")

Parameter(0.34, identity, "\beta", "no description")

In [48]:
# constructor A
Parameter(value, tex) = Parameter(value, identity, tex, "no description")

Parameter

In [49]:
Parameter(0.1, "\beta")

Parameter(0.1, identity, "\beta", "no description")

In [50]:
# constructor B
Parameter(value, transformation)  = Parameter(value, transformation, "notex", "no description")

Parameter

In [ ]:
Parameter(0.1, "\beta")

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mFunction[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:205

In [15]:
# here Julia doesn't know whether it should call constructor A or constructor B

In [55]:
# solution : give different signatures to the various constructors
Parameter(value, tex::String)               = Parameter(value, identity, tex, "no description")
Parameter(value, transformation::Function)  = Parameter(value, transformation, "notex", "no description")

Parameter

In [53]:
Parameter(0.5, "\beta")

Parameter(0.5, identity, "\beta", "no description")

In [54]:
Parameter(0.9, u->u^2)

Parameter(0.9, var"#14#15"(), "notex", "no description")

In [56]:
methods( Parameter )

# 6 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at In[35]:3
[2] Parameter(value, tex::String) in Main at In[55]:2
[3] Parameter(value, transformation::Function) in Main at In[55]:3
[4] Parameter(value, transformation) in Main at In[50]:2
[5] Parameter(value, transformation, tex) in Main at In[45]:1
[6] Parameter(value, transformation, tex_label, description) in Main at In[35]:3

In [8]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Parameter(0.5, identity, "\alpha", "No description available")

Now the function `Parameter` has two different `methods` with different signatures:

In [9]:
methods(Parameter)

# 4 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at In[1]:3
[2] Parameter(value::Float64, tex_label::String) in Main at In[8]:4
[3] Parameter(value, transformation, tex) in Main at In[5]:1
[4] Parameter(value, transformation, tex_label, description) in Main at In[1]:3

In [ ]:
# Find the fields of an instance of a composite type
fieldnames(α)

In [ ]:
α.tex_label

In [ ]:
# Access a particular field using .
α.value

In [ ]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

### Mutable vs non mutable types


by default structures in Julia are non-mutable

In [77]:
β.value = 0.6

LoadError: setfield! immutable struct of type Parameter cannot be changed

In [13]:
mutable struct Params
    x:: Float64
    y:: Float64
end

In [14]:
pos = Params(0.4, 0.2)

Params(0.4, 0.2)

In [15]:
pos.x = 0.5

0.5

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [29]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

In [30]:
Duple(3, -1.0)

LoadError: MethodError: no method matching Duple(::Int64, ::Float64)
[0mClosest candidates are:
[0m  Duple(::T, [91m::T[39m) where T at In[29]:3

In [31]:
Duple{Int64}

Duple{Int64}

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [40]:
Duple(1.3, 3.4)

Truple3{Float64, Int64}((3.4, 5), 4)

In [ ]:
Duple("Hello", "Your")

We can also restrict the type parameter `T` using the type hierarchy.

In [41]:
typeof("S")

String

In [37]:
typeof("S") <: Number

false

In [46]:
typeof(4.6) <: Float64

true

In [47]:
Float64 <: Number

true

In [48]:
# T can be any subtype of Number, but nothing else
struct PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [49]:
PlanarCoordinate("4th Ave", "14th St")

LoadError: MethodError: no method matching PlanarCoordinate(::String, ::String)

In [50]:
PlanarCoordinate(2//3, 8//9)

PlanarCoordinate{Rational{Int64}}(2//3, 8//9)

In [ ]:
Number:

In [53]:
349//80 + 3//4 # rational 

409//80

In [65]:
typeof( factorial(20) )

Int64

In [67]:
typeof( big(20) )

BigInt

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [90]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0:: Integer
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0 :: Float64
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

-25554.110202663698

In [91]:
@time sumofsins_unstable(Int(1e5))

  0.000268 seconds


-25554.110202663698

In [92]:
@time sumofsins_stable(Int(1e5))

  0.000130 seconds


-25554.110202663698

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [51]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

print_type (generic function with 1 method)

In [53]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

print_type (generic function with 2 methods)

In [54]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

print_type (generic function with 3 methods)

In [55]:
# See all methods for a given function
methods(print_type)

# 3 methods for generic function "print_type":
[1] print_type(x::String) in Main at In[53]:3
[2] print_type(x::Number) in Main at In[51]:3
[3] print_type(x::Number, y::Number) in Main at In[54]:3

Julia uses **multiple dispatch** to decide which **method** of a **function** to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [56]:
print_type(5)

5 is a number


In [57]:
print_type("foo")

foo is a string


In [58]:
print_type([1, 2, 3])

MethodError: MethodError: no method matching print_type(::Array{Int64,1})
Closest candidates are:
  print_type(!Matched::String) at In[53]:3
  print_type(!Matched::Number) at In[51]:3
  print_type(!Matched::Number, !Matched::Number) at In[54]:3

#### Other types of functions

Julia supports a short function definition for one-liners

In [ ]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

As well as a special syntax for anonymous functions

In [ ]:
u->u^2

In [ ]:
map(u->u^2, [1,2,3,4])

### Keyword arguments and optional arguments

In [59]:
f(a,b,c=true; algo="newton")

UndefVarError: UndefVarError: f not defined

### Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. _A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently._

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

## Additional Exercises

Taken from QuantEcon's [Julia Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1. Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using `enumerate`, write a function `p` such that `p(x, coeff)` computes the value of the polynomial with coefficients `coeff` evaluated at `x`.

2. Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations.